<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Запросы-к-БД." data-toc-modified-id="Запросы-к-БД.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Запросы к БД.</a></span><ul class="toc-item"><li><span><a href="#Первая-часть-проекта." data-toc-modified-id="Первая-часть-проекта.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Первая часть проекта.</a></span></li><li><span><a href="#Вторая-часть-проекта." data-toc-modified-id="Вторая-часть-проекта.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Вторая часть проекта.</a></span></li></ul></li></ul></div>

# Описание проекта.

В самостоятельном проекте вы будете работать с базой данных StackOverflow — сервиса вопросов и ответов о программировании. StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам.
Вы будете работать с версией базы, где хранятся данные о постах за 2008 год, но в таблицах вы найдёте информацию и о более поздних оценках, которые эти посты получили.

*В данных, которые вы будете использовать в проекте, упоминаются сервисы и компании, запрещённые в РФ.

## Запросы к БД.

### Первая часть проекта. 

**1. Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».**

SELECT COUNT(post_type_id)  
FROM stackoverflow.posts p  
JOIN stackoverflow.post_types pt ON pt.id = p.post_type_id  
WHERE type = 'Question' AND (score>300 OR favorites_count>=100);

**2. Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.**

WITH question AS (  
    SELECT COUNT(post_type_id) AS count_post,  
    p.creation_date::date AS post_dates  
    FROM stackoverflow.posts p  
    JOIN stackoverflow.post_types pt ON pt.id = p.post_type_id  
    WHERE type = 'Question' AND (p.creation_date::date BETWEEN '2008-11-01' AND '2008-11-18')  
    GROUP BY p.creation_date::date)  

SELECT ROUND(AVG(count_post))
FROM question;

**3. Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.**

SELECT COUNT(DISTINCT user_id)  
FROM stackoverflow.badges b  
JOIN stackoverflow.users u ON u.id = b.user_id  
WHERE b.creation_date::date = u.creation_date::date;  

**4. Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?**

SELECT COUNT(DISTINCT(p.id))  
FROM stackoverflow.posts p  
JOIN stackoverflow.users u ON u.id = p.user_id  
JOIN stackoverflow.votes v ON v.post_id = p.id  
WHERE u.display_name = 'Joel Coehoorn';  

**5. Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.**

SELECT *,  
       RANK() OVER (ORDER BY id DESC)  
FROM stackoverflow.vote_types  
ORDER BY id;  

**6. Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.**

SELECT u.id,  
       COUNT(v.id)  
FROM stackoverflow.users u  
JOIN stackoverflow.votes v ON v.user_id = u.id  
JOIN stackoverflow.vote_types vt ON vt.id = v.vote_type_id  
WHERE vt.name = 'Close'  
GROUP BY u.id  
ORDER BY COUNT(v.id) DESC LIMIT 10;  

**7. Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно. Отобразите несколько полей:
идентификатор пользователя;
число значков;
место в рейтинге — чем больше значков, тем выше рейтинг.
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.**

SELECT b.user_id,  
       COUNT(b.id),  
       DENSE_RANK () OVER (ORDER BY COUNT(b.id) DESC)  
FROM stackoverflow.badges b  
JOIN stackoverflow.users u ON b.user_id = u.id  
WHERE b.creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'  
GROUP BY b.user_id  
ORDER BY COUNT(b.id) DESC, b.user_id ASC LIMIT 10;  

**8. Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
заголовок поста;
идентификатор пользователя;
число очков поста;
среднее число очков пользователя за пост, округлённое до целого числа.
Не учитывайте посты без заголовка, а также те, что набрали ноль очков.**

SELECT title,  
       user_id,  
       score,  
       ROUND(AVG(score) OVER (PARTITION BY user_id)) AS avg_score      
FROM stackoverflow.posts  
WHERE title IS NOT NULL AND score != 0;  

**9. Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.**

WITH request AS (SELECT u.id,   
               COUNT(DISTINCT b.user_id)  
FROM stackoverflow.badges b  
JOIN stackoverflow.users u ON u.id = b.user_id        
GROUP BY u.id  
HAVING(COUNT(DISTINCT b.id))>1000)  

SELECT title  
FROM request r  
JOIN stackoverflow.posts AS p ON r.id = p.user_id  
WHERE p.title IS NOT NULL;  

**10. Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:  
пользователям с числом просмотров больше либо равным 350 присвойте группу 1;  
пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;  
пользователям с числом просмотров меньше 100 — группу 3.  
Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу.  
Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.**  

SELECT id,  
       views,  
             CASE  
                 WHEN views >= 350 THEN 1  
                 WHEN views <= 350 AND views >=100 THEN 2  
                 WHEN views <100 THEN 3  
             END    
FROM stackoverflow.users  
WHERE location LIKE '%United States%' and views != 0  
ORDER BY id;  

**11. Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.**

WITH count_cat AS(  
       SELECT id,  
       views,  
             CASE  
                 WHEN views >= 350 THEN 1  
                 WHEN views <= 350 AND views >=100 THEN 2  
                 WHEN views <100 THEN 3  
             END category  
FROM stackoverflow.users  
WHERE location LIKE '%United States%' and views != 0),  

count_2 AS(  
        SELECT id,  
               MAX(views) OVER (PARTITION BY category) AS cat_filtrd,  
               views,  
               category  
FROM count_cat)  


SELECT id,  
       category,  
       cat_filtrd  
FROM count_2  
WHERE views = cat_filtrd  
ORDER BY cat_filtrd DESC, id ASC;  

**12. Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:  
номер дня;  
число пользователей, зарегистрированных в этот день;  
сумму пользователей с накоплением.**  

WITH count_day AS(  
         SELECT EXTRACT (DAY FROM creation_date::date) AS day_cnt,  
         COUNT(id) AS id_cnt  
from stackoverflow.users as u  
WHERE creation_date BETWEEN '2008-11-01' AND '2008-12-01'  
GROUP BY creation_date::date  
ORDER BY creation_date::date)  

SELECT day_cnt,  
       id_cnt,  
       SUM(id_cnt) OVER (ORDER BY day_cnt) AS user_sum  
FROM count_day  
ORDER BY day_cnt;  

**13. Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:  
идентификатор пользователя;  
разницу во времени между регистрацией и первым постом.**  

SELECT u.id,  
       MIN(p.creation_date) - u.creation_date  
FROM stackoverflow.users u  
JOIN stackoverflow.posts p ON p.user_id = u.id  
GROUP BY u.id, u.creation_date;  

### Вторая часть проекта. 

**1. Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.**

SELECT DATE_TRUNC('month', creation_date)::date AS date_posts,  
       SUM(views_count) AS views_cnt  
FROM stackoverflow.posts p  
GROUP BY date_posts  
ORDER BY views_cnt DESC;  

**2. Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.**

SELECT u.display_name,  
       COUNT(DISTINCT p.user_id)  
FROM stackoverflow.users u  
JOIN stackoverflow.posts p ON p.user_id = u.id  
JOIN stackoverflow.post_types pt ON pt.id = p.post_type_id  
WHERE pt.type = 'Answer' AND DATE_TRUNC ('day', p.creation_date) <= DATE_TRUNC ('day', u.creation_date + INTERVAL '1 month')  
GROUP BY u.display_name  
HAVING COUNT(pt.id)>100  
ORDER BY u.display_name;  

**3. Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.**

WITH a AS (SELECT DISTINCT u.id AS id  
FROM stackoverflow.users u   
JOIN stackoverflow.posts p ON u.id=p.user_id    
WHERE CAST(DATE_TRUNC('day', u.creation_date) AS DATE) BETWEEN '2008-09-01' AND '2008-09-30'  
      AND CAST(DATE_TRUNC('day', p.creation_date) AS DATE) BETWEEN '2008-12-01' AND '2008-12-30')  
SELECT CAST(DATE_TRUNC('month', p.creation_date) AS DATE),  
       COUNT(p.id)         
FROM a  
JOIN stackoverflow.posts p ON a.id=p.user_id  
WHERE CAST(DATE_TRUNC('year', p.creation_date) AS DATE) BETWEEN '2008-01-01' AND '2008-12-01'  
GROUP BY CAST(DATE_TRUNC('month', p.creation_date) AS DATE)  
ORDER BY 1 DESC;  

**4. Используя данные о постах, выведите несколько полей:  
идентификатор пользователя, который написал пост;  
дата создания поста;  
количество просмотров у текущего поста;  
сумму просмотров постов автора с накоплением.  
Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же   пользователе — по возрастанию даты создания поста.**  

SELECT DISTINCT user_id,  
       creation_date,  
       views_count,  
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date ASC) AS sum_views  
FROM stackoverflow.posts  
GROUP BY user_id, creation_date, views_count  
ORDER BY user_id ASC;  

**5. Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.**

WITH active_days AS(  
                   SELECT user_id,  
                   COUNT(DISTINCT(CAST (creation_date AS DATE))) AS act_days  
FROM  stackoverflow.posts  
WHERE CAST(creation_date AS DATE) BETWEEN '2008-12-01' AND '2008-12-07'  
GROUP BY user_id)  

SELECT ROUND(AVG(act_days))  
FROM active_days;  

**6. На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:  
номер месяца;  
количество постов за месяц;  
процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.  
Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение   процента до двух знаков после запятой.  
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до   ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.**    

WITH month_cnt AS(  
               SELECT DATE_PART('month', creation_date) AS month_post,  
               COUNT(id) AS post_cnt  
               FROM stackoverflow.posts  
               WHERE creation_date >= '2008-09-01' AND creation_date <= '2008-12-31'  
               GROUP BY 1)  
              
SELECT month_post,  
       post_cnt,  
       ROUND(100 *(post_cnt - LAG(post_cnt) OVER (ORDER BY month_post))::NUMERIC/ LAG(post_cnt) OVER (ORDER BY month_post),   2) AS post_change  
FROM month_cnt;  

**7. Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:  
номер недели;  
дата и время последнего поста, опубликованного на этой неделе.**    

WITH top_user as ( SELECT   
EXTRACT(WEEK FROM DATE_TRUNC('week',creation_date::date)) as week_number,   
creation_date   
FROM stackoverflow.posts   
WHERE user_id = 22656 and creation_date::date between '2008-10-01' and '2008-10-31')   
 
SELECT DISTINCT week_number,   
MAX(creation_date)OVER(PARTITION BY week_number)   
FROM top_user   
ORDER BY week_number;  